# R-Kernel required.

# Define reader-functions (parsing)

In [1]:
read_flow_data <- function(event_number, basin_number ){
    path <- paste('ERAD files/flow/Ev', c(event_number), sep="")
    path <- paste(path, c(basin_number), sep="_")
    path <- paste(path, 'csv', sep='.')
    data <- read.csv(file=path, header=FALSE, sep=',')
    data$date <- paste(data$V1,data$V2,data$V3,data$V4,data$V5)
    data$date <- as.POSIXlt(data$date, tz="", format="%Y %m %d %H %M")
    data <- data[c("V6", "date")]
    data$V6 <- data$V6*0.0283168466 # conversion to m^3/s
    return(data)
}

read_basin_data <- function(event_number, basin_number, resolution){
    path <- paste('ERAD files/Rainfall_BA/ERAD_BA_', c(basin_number), sep="")
    path <- paste(path, c(event_number), sep="_E")
    path <- paste(path, c(resolution), sep="_")
    path <- paste(path, 'min.csv', sep='')
    data = read.csv(file=path, header=TRUE, sep=',')
    data$date <- paste(data$X0,data$X1,data$X2,data$X3,data$X4)
    data$date <- as.POSIXlt(data$date, tz="", format="%Y %m %d %H %M")
    data <- data[c("BAverage", "date")]
    return(data)
}

read_gauge_data <- function(event_number, basin_number, resolution, gauge_number){
    path <- paste('ERAD files/RG/ERAD_RG_', c(basin_number), sep="")
    path <- paste(path, c(event_number), sep="_E")
    path <- paste(path, c(resolution), sep="_")
    path <- paste(path, 'min.csv', sep='')
    data = read.csv(file=path, header=TRUE, sep=',')
    data$date <- paste(data$X0,data$X1,data$X2,data$X3,data$X4)
    data$date <- as.POSIXlt(data$date, tz="", format="%Y %m %d %H %M")
    data <- data[c(gauge_number, "date")]
    names(data)[names(data) == gauge_number] <- 'RG'
    return(data)
}

# Read event data of 5 events in basin 409

In [2]:
flow_409_list <- list()
basin15min_409 <- list()
basin60min_409 <- list()
rg15min_409 <- list()
rg60min_409 <- list()
for (i in 1:5) {
    flow_409_list[[i]] = read_flow_data(i, 409)
    basin15min_409[[i]] = read_basin_data(i, 409, 15)
    basin60min_409[[i]] = read_basin_data(i, 409, 60)
    rg15min_409[[i]] = read_gauge_data(i, 409, 15, 'RG_9')
    rg60min_409[[i]] = read_gauge_data(i, 409, 60, 'RG_9')
}

flow_507_list <- list()
basin15min_507 <- list()
basin60min_507 <- list()
rg15min_507 <- list()
rg60min_507 <- list()
for (i in 1:5) {
    flow_507_list[[i]] = read_flow_data(i, 507)
    basin15min_507[[i]] = read_basin_data(i, 507, 15)
    basin60min_507[[i]] = read_basin_data(i, 507, 60)
    rg15min_507[[i]] = read_gauge_data(i, 507, 15, 'RG_66')
    rg60min_507[[i]] = read_gauge_data(i, 507, 60, 'RG_66')
}

# Define plotting functions

In [3]:
Sys.setenv("LANGUAGE"="En")
Sys.setlocale("LC_ALL", "en_US.UTF-8")

plot_flow_basin_avg_overlapping <- function(out_file_name, ba_events_15min, ba_events_60min, flow_events, title_basin_id){
    pdf(out_file_name, width=10, height=22)
    par(mfrow=c(5,1),oma=c(0,0,2,0)) 
    par(xaxs="i", yaxs="i", mar=c(5,5,5,5))
    for (i in 1:length(ba_events_15min)) {
        plot(ba_events_15min[[i]]$date, ba_events_15min[[i]]$BAverage, type="s", ylim=c(max(ba_events_15min[[i]]$BAverage)*1.5,0),
         axes=FALSE, xlab=NA, ylab=NA, col="blue", lwd=2, lend="square")
        lines(ba_events_60min[[i]]$date, ba_events_60min[[i]]$BAverage/4, type="s", ylim=c(max(ba_events_15min[[i]]$BAverage)*1.5,0), xlab=NA, ylab=NA, col="red", lwd=2, lend="square")
        legend("bottomright", c("basinAVG 15-min", "basinAVG 60-min"), lwd=2, col=c("blue", "red"))
        axis(4)
        mtext("Rainfall [mm/15min]", side=4, line=3)
        par(new=TRUE)
        plot(flow_events[[i]]$date, flow_events[[i]]$V6, type="l", lwd=2, ylim=c(0, max(flow_events[[i]]$V6)*1.2), ylab='Flow [m^3/s]', xlab='')
        cur_title <- paste('Basin-average rainfall vs flow for Event ', i)
        cur_title <- paste(cur_title, ' at ')
        cur_title <- paste(cur_title, title_basin_id)
        title(cur_title)
    }
    dev.off()
    }

plot_flow_gauge_overlapping <- function(out_file_name, rg_events_15min, rg_events_60min, flow_events, title_basin_id){
    pdf(out_file_name, width=10, height=22)
    par(mfrow=c(5,1)) 
    par(xaxs="i", yaxs="i", mar=c(5,5,5,5))
    for (i in 1:length(rg_events_15min)) {
        plot(rg_events_15min[[i]]$date, rg_events_15min[[i]]$RG, type="s", ylim=c(max(rg_events_15min[[i]]$RG)*1.5,0),
         axes=FALSE, xlab=NA, ylab=NA, col="blue", lwd=2, lend="square")
        lines(rg_events_60min[[i]]$date, rg_events_60min[[i]]$RG/4, type="s", ylim=c(max(rg_events_15min[[i]]$RG)*1.5,0),
              xlab=NA, ylab=NA, col="red", lwd=2, lend="square")
        legend("bottomright", c("RG 15-min", "RG 60-min"), lwd=2, col=c("blue", "red"))
        axis(4)
        mtext("Rainfall [mm/15min]", side=4, line=3)
        par(new=TRUE)
        plot(flow_events[[i]]$date, flow_events[[i]]$V6, type="l", lwd=2, ylim=c(0, max(flow_events[[i]]$V6)*1.2), ylab='Flow [m^3/s]', xlab='')        
        cur_title <- paste('Gauge measured rainfall vs flow for Event ', i)
        cur_title <- paste(cur_title, ' at ')
        cur_title <- paste(cur_title, title_basin_id)
        title(cur_title)
    }
    dev.off()
    }

[1] "LC_CTYPE=en_US.UTF-8;LC_NUMERIC=C;LC_TIME=en_US.UTF-8;LC_COLLATE=en_US.UTF-8;LC_MONETARY=en_US.UTF-8;LC_MESSAGES=en_US.UTF-8;LC_PAPER=nl_NL.UTF-8;LC_NAME=C;LC_ADDRESS=C;LC_TELEPHONE=C;LC_MEASUREMENT=nl_NL.UTF-8;LC_IDENTIFICATION=C"

## Basin average plot

In [4]:
plot_flow_basin_avg_overlapping('flow_BAOverlapping_409.pdf', basin15min_409, basin60min_409, flow_409_list, '409')
plot_flow_basin_avg_overlapping('flow_BAOverlapping_507.pdf', basin15min_507, basin60min_507, flow_507_list, '507')

png 
  2

png 
  2

## RG plot

In [5]:
plot_flow_gauge_overlapping('flow_RGOverlapping_409.pdf', rg15min_409, rg60min_409, flow_409_list, '409')
plot_flow_gauge_overlapping('flow_RGOverlapping_507.pdf', rg15min_507, rg60min_507, flow_507_list, '507')

png 
  2

png 
  2

## Runoff Ratio (RR)

In [6]:
print_runoff_ratio <- function(flow_ev, ba_rain_15min, basin_area) {
    flow_ev$V6 <- flow_ev$V6 * 1000 * 60/ basin_area
    flow_15_min <- aggregate(flow_ev[1], list(cut(flow_ev[[2]], '15 mins')), sum)
    rr <- sum(flow_15_min[[2]]) / sum(ba_rain_15min)
    print(rr)
    return(rr)
}

In [7]:
sum(basin15min_409[[i]][[1]]/4)

[1] 2.6647

In [8]:
runoff_holder_basin_15_409 <- list()
print('Runoff basin 15 409')
for (i in 1:5) {
    temp <- basin15min_409[[i]][[1]]
    rr <- print_runoff_ratio(flow_409_list[[i]], temp, 31*1000000)
    runoff_holder_basin_15_409[[i]] <- rr
}

runoff_holder_basin_15_507 <- list()
print('Runoff basin 15 507')
for (i in 1:5) {
    temp <- basin15min_507[[i]][[1]]
    rr <- print_runoff_ratio(flow_507_list[[i]], temp, 111* 1000000)
    runoff_holder_basin_15_507[[i]] <- rr
}


[1] "Runoff basin 15 409"
[1] 0.6268238
[1] 0.4730038
[1] 0.5317724
[1] 0.4380284
[1] 0.9333338
[1] "Runoff basin 15 507"
[1] 0.6457988
[1] 0.5529407
[1] 0.514497
[1] 0.6114855
[1] 1.25599


In [9]:
runoff_holder_409_rg <- list()
print('Runoff RG 409')
for (i in 1:5) {
    rr <- print_runoff_ratio(flow_409_list[[i]], rg15min_409[[i]][[1]], 31*1000000)
    runoff_holder_409_rg[[i]] <- rr
}

runoff_holder_507_rg <- list()
print('Runoff RG 507')
for (i in 1:5) {
    rr <- print_runoff_ratio(flow_507_list[[i]], rg15min_507[[i]][[1]], 111*1000000)
    runoff_holder_507_rg[[i]] <- rr
}

[1] "Runoff RG 409"
[1] 0.5659042
[1] 0.6081782
[1] 0.6013829
[1] 0.5742808
[1] 0.3035199
[1] "Runoff RG 507"
[1] 0.3948349
[1] 0.5719734
[1] 0.415323
[1] 0.4608634
[1] 0.5118758


## Response Time (RT)

In [10]:
print_response_time <- function(flow, rain) {
    idx_max_flow <- which.max(flow[[1]])
    date_max_flow <- flow$date[idx_max_flow]
    
    idx_max_rain <- which.max(rain[[1]])
    date_max_rain <- rain$date[idx_max_rain]
    
    print(date_max_flow - date_max_rain)
    rt <- date_max_flow - date_max_rain
    rt <- as.numeric(rt, units="mins")
    return(rt)
}

## Basin 

In [11]:
rt_holder_basin_15_409 <- list()
rt_holder_basin_60_409 <- list()
for (i in 1:5) {
    print('15 min:')
    rt <- print_response_time(flow_409_list[[i]], basin15min_409[[i]])
    rt_holder_basin_15_409[[i]] <- rt
    print('60 min:')
    rt <- print_response_time(flow_409_list[[i]], basin60min_409[[i]])
    rt_holder_basin_60_409[[i]] <- rt
}

[1] "15 min:"
Time difference of 1.316667 hours
[1] "60 min:"
Time difference of 1.816667 hours
[1] "15 min:"
Time difference of 1.15 hours
[1] "60 min:"
Time difference of 1.15 hours
[1] "15 min:"
Time difference of 42 mins
[1] "60 min:"
Time difference of 1.2 hours
[1] "15 min:"
Time difference of 1.15625 days
[1] "60 min:"
Time difference of 1.166667 days
[1] "15 min:"
Time difference of 15 mins
[1] "60 min:"
Time difference of 1.25 hours


In [12]:
rt_holder_basin_15_507 <- list()
rt_holder_basin_60_507 <- list()
for (i in 1:5) {
    print('15 min:')
    rt <- print_response_time(flow_507_list[[i]], basin15min_507[[i]])
    rt_holder_basin_15_507[[i]] <- rt
    print('60 min:')
    rt <- print_response_time(flow_507_list[[i]], basin60min_507[[i]])
    rt_holder_basin_60_507[[i]] <- rt
}

[1] "15 min:"
Time difference of 2.216667 hours
[1] "60 min:"
Time difference of 1.966667 hours
[1] "15 min:"
Time difference of 2.8 hours
[1] "60 min:"
Time difference of 2.8 hours
[1] "15 min:"
Time difference of 2.333333 hours
[1] "60 min:"
Time difference of 2.833333 hours
[1] "15 min:"
Time difference of 1.197917 days
[1] "60 min:"
Time difference of 1.208333 days
[1] "15 min:"
Time difference of 1 hours
[1] "60 min:"
Time difference of 2 hours


## Gauge

In [13]:
rt_holder_rg_15_409 <- list()
rt_holder_rg_60_409 <- list()
for (i in 1:5) {
    print('15 min:')
    rt <- print_response_time(flow_409_list[[i]], rg15min_409[[i]])
    rt_holder_rg_15_409[[i]] <- rt
    print('60 min:')
    rt <- print_response_time(flow_409_list[[i]], rg60min_409[[i]])
    rt_holder_rg_60_409[[i]] <- rt
}

[1] "15 min:"
Time difference of 19 mins
[1] "60 min:"
Time difference of 1.816667 hours
[1] "15 min:"
Time difference of 1.15 hours
[1] "60 min:"
Time difference of 1.15 hours
[1] "15 min:"
Time difference of 42 mins
[1] "60 min:"
Time difference of 1.2 hours
[1] "15 min:"
Time difference of -17.75 hours
[1] "60 min:"
Time difference of 1.166667 days
[1] "15 min:"
Time difference of 15 mins
[1] "60 min:"
Time difference of 15 mins


In [14]:
rt_holder_rg_15_507 <- list()
rt_holder_rg_60_507 <- list()
for (i in 1:5) {
    print('15 min:')
    rt <- print_response_time(flow_507_list[[i]], rg15min_507[[i]])
    rt_holder_rg_15_507[[i]] <- rt
    print('60 min:')
    rt <- print_response_time(flow_507_list[[i]], rg60min_507[[i]])
    rt_holder_rg_60_507[[i]] <- rt
}

[1] "15 min:"
Time difference of 2.216667 hours
[1] "60 min:"
Time difference of 2.966667 hours
[1] "15 min:"
Time difference of 3.55 hours
[1] "60 min:"
Time difference of 2.8 hours
[1] "15 min:"
Time difference of 2.333333 hours
[1] "60 min:"
Time difference of 2.833333 hours
[1] "15 min:"
Time difference of -16.5 hours
[1] "60 min:"
Time difference of 1.208333 days
[1] "15 min:"
Time difference of 1.25 hours
[1] "60 min:"
Time difference of 1 hours


# Plot the results

### Runoff Ratio

In [15]:
x <- rep(1, 5)
x <- c(x, rep(2, 5))
x <- c(x, rep(3, 5))
x <- c(x, rep(4, 5))
rr_list <- c( runoff_holder_basin_15_409, runoff_holder_409_rg, runoff_holder_basin_15_507, runoff_holder_507_rg )  

In [16]:
pdf('runoff.pdf', width=7, height=5)
plot(x, rr_list, xaxt='n', ylab='Runoff Ratio (RR) [-]', xlab='Dataset')
title('Runoff Ratio for basins 409 and 507')
abline(v=2.5, col="blue")
axis(1,at=c(1, 2, 3, 4),labels=c("Basin15 409", "RG15 409", "Basin15 507", "RG15 507"))
dev.off()

png 
  2

### Response Time

In [17]:
x <- rep(1, 5) # there's probably a better way to do that in R ...
x <- c(x, rep(2, 5))
x <- c(x, rep(3, 5))
x <- c(x, rep(4, 5))
x <- c(x, rep(5, 5))
x <- c(x, rep(6, 5))
x <- c(x, rep(7, 5))
x <- c(x, rep(8, 5))
rt_list <- c(rt_holder_basin_15_409, rt_holder_basin_60_409, rt_holder_rg_15_409, rt_holder_rg_60_409, rt_holder_basin_15_507, rt_holder_basin_60_507, rt_holder_rg_15_507, rt_holder_rg_60_507)

In [18]:
pdf('response_time.pdf', width=14, height=5)
#plot(x, rt_list, xaxt='n', ylab='Response Time [min]', xlab='Dataset', log='y')
#idx <- rt_list<0
#plot(x[!idx], rt_list[!idx], xaxt='n', ylab='Response Time [min]', xlab='Dataset', log='y', col='black')
#points(x[idx], lapply(rt_list[idx], abs), col='red')

plot(x, rt_list, xaxt='n', ylab='Response Time [min]', xlab='Dataset', col='black')

title('Response Time for basins 409 and 507')
abline(v=4.5, col="blue")
axis(1,at=c(1, 2, 3, 4, 5, 6, 7, 8),labels=c("Basin15 409", "Basin60 409", "RG15 409", "RG60 409", "Basin15 507", "Basin60 507", "RG15 507", "RG60 507"))
dev.off()

png 
  2